<a href="https://colab.research.google.com/github/parharti/GSoC_prework/blob/main/GSoC_prework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy nibabel scikit-image matplotlib opencv-python torch torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

**Code to generate synthetic data**


In [ ]:
import numpy as np

shape = (128, 128, 64, 3)
synthetic_data = np.random.normal(loc=100, scale=25, size=shape)

for ch in range(3):
    for _ in range(10):  # Add 20 blobs
        x, y, z = np.random.randint(20, 108), np.random.randint(20, 108), np.random.randint(10, 54)
        rr, cc, zz = np.meshgrid(
            np.arange(x-3, x+4),
            np.arange(y-3, y+4),
            np.arange(z-2, z+3)
        )
        synthetic_data[rr, cc, zz, ch] += 100

# Save as .npy file
output_npy_path = "synthetic_brain_3ch.npy"
np.save(output_npy_path, synthetic_data.astype(np.float32))

output_npy_path


'synthetic_brain_3ch.npy'

**WEEK 1-2 (*early GSoC Prep*)**

✅ Channel-wise detection

✅ Feature normalization

✅ Combined feature representation

✅ Visualized overlays of blobs

# **2D slice-based blob detection**

In [ ]:
import numpy as np
from skimage.feature import blob_log
from skimage.color import rgb2gray
import matplotlib.pyplot as plt


def detect_blobs(img_2d, min_sigma=1, max_sigma=5, num_sigma=10, threshold=0.5):
    blobs = blob_log(img_2d, min_sigma=min_sigma, max_sigma=max_sigma, num_sigma=num_sigma, threshold=threshold)
    return blobs

def normalize_features(features, method='z-score'):
    if method == 'z-score':
        mean = np.mean(features)
        std = np.std(features)
        return (features - mean) / std

    return features

def extract_and_combine_features(img, num_channels=3):
    all_features = []

    for c in range(num_channels):
        channel = img[:, :, 50, c]
        blobs = detect_blobs(channel)
        all_features.append(blobs)


        feature_sizes = np.array([blob[2] for blob in blobs])
        normalized_sizes = normalize_features(feature_sizes)

        all_features[c] = (blobs, normalized_sizes)


    combined_features = np.concatenate([feature[1] for feature in all_features], axis=0)

    return all_features, combined_features

def plot_blobs(image, blobs, title=""):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(image, cmap='gray')
    for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color='red', linewidth=1, fill=False)
        ax.add_patch(c)
    ax.set_title(title)
    plt.axis('off')
    plt.show()


img = np.load('/content/synthetic_brain_3ch.npy')
num_channels = 3


all_features, combined_features = extract_and_combine_features(img, num_channels)


channel_1 = img[:, :, 50, 0]
blobs_1 = all_features[0][0]



print("Combined Features (Normalized):", combined_features)


Combined Features (Normalized): [ 7.3795786  -0.14496204 -0.14496204 ... -0.12535805 -0.12535805
 -0.12535805]


# **Full 3D volume-based detection**

In [ ]:
import numpy as np
from skimage.feature import blob_log
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


def detect_blobs_3d(volume_3d, min_sigma=1, max_sigma=5, num_sigma=10, threshold=0.2):
    all_blobs = []
    for z in range(volume_3d.shape[2]):
        slice_2d = volume_3d[:, :, z]
        blobs = blob_log(slice_2d, min_sigma=min_sigma, max_sigma=max_sigma,
                         num_sigma=num_sigma, threshold=threshold)
        for blob in blobs:
            y, x, r = blob
            all_blobs.append([z, y, x, r])
    return np.array(all_blobs)

def normalize_features(features, method='z-score'):
    if method == 'z-score':
        mean = np.mean(features)
        std = np.std(features)
        return (features - mean) / std
    return features

def extract_and_combine_features_3d(img, num_channels=3):
    all_features = []

    for c in range(num_channels):
        print(f"Processing channel {c + 1}/{num_channels}")
        volume = img[:, :, :, c]
        blobs = detect_blobs_3d(volume)

        if blobs.size == 0:
            print(f"No blobs found in channel {c}")
            all_features.append((np.array([]), np.array([])))
            continue

        feature_sizes = np.array([blob[3] for blob in blobs])
        normalized_sizes = normalize_features(feature_sizes)
        all_features.append((blobs, normalized_sizes))

    combined_features = np.concatenate(
        [feature[1] for feature in all_features if feature[1].size > 0], axis=0)

    return all_features, combined_features

def plot_blobs_3d(blobs, title="3D Blobs"):
    if blobs.size == 0:
        print("No blobs to plot.")
        return

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    z, y, x, r = blobs[:, 0], blobs[:, 1], blobs[:, 2], blobs[:, 3]
    ax.scatter(x, y, z, s=r * 10, c='red', alpha=0.5)
    ax.set_title(title)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    plt.show()



img = np.load('/content/synthetic_brain_3ch.npy')
num_channels = img.shape[3]

all_features, combined_features = extract_and_combine_features_3d(img, num_channels)


channel_1_blobs = all_features[0][0]


# Print combined normalized features
print("Combined Normalized Features:", combined_features)


Processing channel 1/3
Processing channel 2/3
Processing channel 3/3
Combined Normalized Features: [-0.12622162 -0.12622162 -0.12622162 ... -0.12699761 -0.12699761
 -0.12699761]
